## 4. Feature_Engineering

In [3]:
#Import libraries for feature engineering
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_regression, SelectKBest
import warnings

# Configure settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("🔧 FEATURE ENGINEERING SETUP")
print("=" * 35)
print("✅ Libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🤖 Scikit-learn available for feature selection")

🔧 FEATURE ENGINEERING SETUP
✅ Libraries imported successfully!
📊 Pandas version: 2.2.3
🤖 Scikit-learn available for feature selection


In [115]:
#Load cleaned data (from data processing step)
df = pd.read_csv('../data/processed/prepocessed_daily_data.csv')  # Will be updated to use processed data
df.set_index('datetime', inplace=True)
df.index = pd.to_datetime(df.index)

In [117]:
df.columns

Index(['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'windgust', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'sunrise',
       'sunset', 'moonphase', 'year', 'month', 'day', 'day_of_year', 'season',
       'temp_range'],
      dtype='object')

In [118]:
df['sunrise'] = pd.to_datetime(df['sunrise'])
df['sunset'] = pd.to_datetime(df['sunset'])
df['day_length_hours'] = df['sunset'] - df['sunrise']
df = df.drop(columns=['sunrise', 'sunset'])
df['day_length_hours'] = df['day_length_hours'].dt.total_seconds() / 3600.0

df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['dayofyear_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['dayofyear_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)

df.drop(columns=['day', 'month', 'day_of_year'], inplace=True)

df['target5+'] = df['temp'].shift(-5)
df['target4+'] = df['temp'].shift(-4)
df['target3+'] = df['temp'].shift(-3)
df['target2+'] = df['temp'].shift(-2)
df['target1+'] = df['temp'].shift(-1)
df = df[~df['target5+'].isna()].copy()

In [119]:
df["temp_solar_interaction"] = df["temp"] * df["solarradiation"]
df["uv_temp_interaction"] = df["uvindex"] * df["temp"]
df['temp_cloudcover_interaction'] = df['temp'] * df['cloudcover']
df['temp_sealevelpressure_interaction'] = df['temp'] * df['sealevelpressure']
df['weighted_precip'] = df['precipprob'] * df['precip']
df['effective_solar'] = df['solarradiation'] * (1 - df['cloudcover']/100)
df['precip_impact'] = df['precipprob'] * df['precip']

In [120]:
df['wind_u'] = df['windspeed'] * np.cos(2 * np.pi * df['winddir'] / 360)  # gió đông-tây
df['wind_v'] = df['windspeed'] * np.sin(2 * np.pi * df['winddir'] / 360)  # gió nam-bắc
df = df.drop('winddir', axis=1)

temp_minus_dew = df['temp'] - df['dew']

# Tạo feature moonphase_sin
df['moonphase_sin'] = np.sin(2 * np.pi * df['moonphase'] / 1)

# Tạo feature moonphase_cos
df['moonphase_cos'] = np.cos(2 * np.pi * df['moonphase'] / 1)

# (Tùy chọn) Bỏ cột moonphase gốc
df = df.drop('moonphase', axis=1)

In [121]:
# Create lagging features
def create_lag_features(df, cols, lags):
    for col in cols:
        for lag in lags:
            df[f"{col}_lag_{lag}"] = df[col].shift(lag)
    return df

# Specify columns and lags
# Get all numerical columns
computing_columns = df.drop(columns=['year', 'season', 'target5+', 'target4+', 'target3+', 'target2+', 'target1+', 'month_sin',
                                    'month_cos', 'dayofyear_sin', 'dayofyear_cos']).columns

lag_steps = [1, 2, 3, 5, 7, 10, 14, 21, 30]  # Example lag steps

# Apply lagging features before handling rolling horizons
df = create_lag_features(df, computing_columns, lag_steps)

# Function to compute rolling mean and percentage change
def compute_rolling(df, horizon, col):
    label = f"rolling_{horizon}_{col}"
    df[label] = df[col].rolling(horizon, min_periods=horizon).mean()  # Ensure full horizon is used
    df[f"{label}_change"] = df[col] - df[label]
    return df

# Compute rolling features for specified horizons
rolling_horizons = [3, 7, 14, 21, 30]  # Rolling windows of 3, 7, 14 days
for horizon in rolling_horizons:
    for col in computing_columns:
        df = compute_rolling(df, horizon, col)

In [ ]:
#Months and days average
def expand_mean(df):
    return df.expanding(1).mean()

for col in computing_columns:
    df[f"month_avg_{col}"] = df[col].groupby(df.index.month, group_keys=False).apply(expand_mean)
    df[f"day_avg_{col}"] = df[col].groupby(df.index.day_of_year, group_keys=False).apply(expand_mean)
    df[f"year_avg_{col}"] = df[col].groupby(df.index.year, group_keys=False).apply(expand_mean)
    df[f"season_avg_{col}"] = df[col].groupby(df['season'], group_keys=False).apply(expand_mean)
    df["month_max_temp"] = df['temp'].groupby(df.index.month, group_keys=False).cummax()
    df["month_min_temp"] = df['temp'].groupby(df.index.month, group_keys=False).cummin()

df["temp_volatility_7"] = df["temp"].rolling(7).std()
df["temp_volatility_14"] = df["temp"].rolling(14).std()
df["temp_volatility_21"] = df["temp"].rolling(21).std()
df["temp_volatility_30"] = df["temp"].rolling(30).std()
df["temp_spike_flag"] = (df["temp"] - df["temp"].shift(1)).abs() > 5
df["temp_anomaly_vs_month_avg"] = df["temp"] - df["month_avg_temp"]
df["temp_anomaly_vs_season_avg"] = df["temp"] - df["season_avg_temp"]

df["pressure_trend_3d"] = df["sealevelpressure"] - df["sealevelpressure"].shift(3)
df["pressure_trend_7d"] = df["sealevelpressure"] - df["sealevelpressure"].shift(30)

In [124]:
# Drop rows with NaN values caused by rolling horizons
df = df.iloc[30:]
# Verify no NaN values exist
nan_summary = df.isna().sum()
print("Summary of NaN values in each column after handling rolling horizons and lagging:")
print(nan_summary[nan_summary > 0].sort_values())

if df.isna().any().any():
    print("\nThe dataframe contains NaN values.")
else:
    print("\nThe dataframe does not contain any NaN values.")

Summary of NaN values in each column after handling rolling horizons and lagging:
Series([], dtype: int64)

The dataframe does not contain any NaN values.


In [125]:
df.shape

(3619, 790)

In [126]:
df.to_csv('../data/processed/feature_engineering_daily_data2.csv')